In [3]:
import sqlite3
import random
from datetime import datetime, timedelta

In [4]:
class Database:
    def __init__(self, db_name):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()

    def create_tables(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Authors (
                author_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                birth_date DATE
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Publishers (
                publisher_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                address TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Books (
                book_id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                author_id INTEGER,
                publisher_id INTEGER,
                publication_year INTEGER,
                FOREIGN KEY (author_id) REFERENCES Authors (author_id),
                FOREIGN KEY (publisher_id) REFERENCES Publishers (publisher_id)
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS Readers (
                reader_id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                address TEXT,
                phone_number TEXT
            )
        ''')

        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS BookLoans (
                loan_id INTEGER PRIMARY KEY AUTOINCREMENT,
                book_id INTEGER,
                reader_id INTEGER,
                loan_date DATE,
                return_date DATE,
                FOREIGN KEY (book_id) REFERENCES Books (book_id),
                FOREIGN KEY (reader_id) REFERENCES Readers (reader_id)
            )
        ''')

        self.conn.commit()

    def insert_sample_data(self):
        # Вставка данных в таблицу Authors
        authors = [('Лев Толстой', '1828-09-09'), ('Фёдор Достоевский', '1821-11-11'), ('Антон Чехов', '1860-01-29')]
        self.cursor.executemany('INSERT INTO Authors (name, birth_date) VALUES (?, ?)', authors)

        # Вставка данных в таблицу Publishers
        publishers = [('Издательство АСТ', 'Москва'), ('Эксмо', 'Москва'), ('Дрофа', 'Санкт-Петербург')]
        self.cursor.executemany('INSERT INTO Publishers (name, address) VALUES (?, ?)', publishers)

        # Вставка данных в таблицу Books
        books = [('Война и мир', 1, 1, 1869), ('Преступление и наказание', 2, 2, 1866), ('Чайка', 3, 3, 1896)]
        self.cursor.executemany('INSERT INTO Books (title, author_id, publisher_id, publication_year) VALUES (?, ?, ?, ?)', books)

        # Вставка данных в таблицу Readers
        readers = [('Иван Иванов', 'Москва', '1234567890'), ('Петр Петров', 'Санкт-Петербург', '0987654321')]
        self.cursor.executemany('INSERT INTO Readers (name, address, phone_number) VALUES (?, ?, ?)', readers)

        # Вставка данных в таблицу BookLoans
        for i in range(1, 4):
            loan_date = datetime.now() - timedelta(days=random.randint(1, 30))
            return_date = loan_date + timedelta(days=random.randint(1, 30))
            self.cursor.execute('INSERT INTO BookLoans (book_id, reader_id, loan_date, return_date) VALUES (?, ?, ?, ?)',
                                (i, random.randint(1, 2), loan_date.strftime('%Y-%m-%d'), return_date.strftime('%Y-%m-%d')))

        self.conn.commit()

    def get_books_with_authors_and_publishers(self):
        self.cursor.execute('''
            SELECT Books.title, Authors.name, Publishers.name
            FROM Books
            JOIN Authors ON Books.author_id = Authors.author_id
            JOIN Publishers ON Books.publisher_id = Publishers.publisher_id
        ''')
        return self.cursor.fetchall()

    def get_loans_with_books_and_readers(self):
        self.cursor.execute('''
            SELECT Books.title, Readers.name, BookLoans.loan_date, BookLoans.return_date
            FROM BookLoans
            JOIN Books ON BookLoans.book_id = Books.book_id
            JOIN Readers ON BookLoans.reader_id = Readers.reader_id
        ''')
        return self.cursor.fetchall()

    def close(self):
        self.conn.close()



In [5]:

db = Database('library.db')
db.create_tables()
db.insert_sample_data()

print("Книги с авторами и издательствами:")
for book in db.get_books_with_authors_and_publishers():
    print(book)

print("\nВыдачи книг с читателями:")
for loan in db.get_loans_with_books_and_readers():
    print(loan)

db.close()

Книги с авторами и издательствами:
('Война и мир', 'Лев Толстой', 'Издательство АСТ')
('Преступление и наказание', 'Фёдор Достоевский', 'Эксмо')
('Чайка', 'Антон Чехов', 'Дрофа')
('Война и мир', 'Лев Толстой', 'Издательство АСТ')
('Преступление и наказание', 'Фёдор Достоевский', 'Эксмо')
('Чайка', 'Антон Чехов', 'Дрофа')

Выдачи книг с читателями:
('Война и мир', 'Иван Иванов', '2024-12-05', '2024-12-30')
('Преступление и наказание', 'Петр Петров', '2024-12-13', '2025-01-10')
('Чайка', 'Иван Иванов', '2024-12-18', '2024-12-25')
('Война и мир', 'Петр Петров', '2024-11-26', '2024-11-29')
('Преступление и наказание', 'Петр Петров', '2024-12-15', '2025-01-03')
('Чайка', 'Иван Иванов', '2024-12-02', '2024-12-30')
